# Lattice based cryptography

In [1]:
import numpy as np
from SVP_utils import SVPoint, read_basis, gram_schmidt, generate_basis_vectors, generate_random_vectors, augment, find_difference, find_modified_average, find_random, write_output
import random

## 1 Read basis
The initial basis is stored here but the basis can also be read from a file.

In [2]:
B = np.array([
 [37, 20, 96, 20, 34, 64, 82, 56, 47, 21, 50, 49],
 [39, 24, 19, 49, 82, 97, 88, 84, 41, 51, 36, 74],
 [19, 56, 37, 73,  4, 12, 72, 18, 46,  8, 54, 94],
 [13, 46, 26,  8, 83, 71, 45, 84, 21, 32, 53, 80],
 [65, 39, 25, 56, 52, 44, 84, 30, 69, 33, 13,  5],
 [59, 56, 90,  1, 42, 58, 90, 92,  2,  6,  7, 80],
 [18, 14, 26, 31, 91, 93, 77, 64, 95, 36, 23,  5],
 [11, 58, 22, 51, 90, 13, 93, 43, 21, 81, 12, 77],
 [42, 65, 99,  6, 23, 43, 94, 30, 37, 66, 34, 66],
 [99, 31, 24, 44, 18, 58, 17, 27, 70, 88, 59, 11],
 [30, 43, 21, 70, 48, 47, 13, 93, 94, 48, 69, 58],
 [ 7, 12, 94, 88, 59, 95, 43, 62, 71, 36, 91, 70]
])

In [3]:
B = read_basis('../task/latticeBasis.txt')

## 2 Brief analysis of the basis vectors

In [4]:
basis_norms = np.array([np.linalg.norm(b) for b in B])
print('average:', sum(basis_norms)/len(basis_norms))
print('variance:', np.std(basis_norms))

# Upper bound on the length of the shortest vector by minkowski's theorem
n = len(B[0])
print(f"Upper bound by Minkowski's theorem: {np.sqrt(n) * (np.linalg.det(B) ** (1/n))}")


# Estimation of the length of the shortest vector by https://www.latticechallenge.org/svp-challenge/#
from math import factorial

g_term = factorial( (n/2+1) -1 ) ** (1/n)
print(f"Estimation from latticechallenge.org: {1.05 * g_term / np.sqrt(np.pi) * (np.linalg.det(B) ** (1/n))}" )

average: 195.73339244318618
variance: 17.522055131034225
Upper bound by Minkowski's theorem: 230.47420094416253
Estimation from latticechallenge.org: 68.1957168688578


## 3 Sieving

In [5]:
def sieve(B):
    """
    Function to sieve for short vectors on a lattice with basis B
    
    Parameters:
    B (np.array): Input lattice basis vectors
    
    Returns:
    SVPoint: the shortest vector found on the lattice
    """
    # 0 Orthogonalise the basis with gram-schmidt
    # B_gs = gram_schmidt(B)

    # 1 Generate n vectors on the lattice
    n = 150

    # vectors = generate_basis_vectors(B, n)
    vectors = generate_random_vectors(B, n, l=-4, h=5)

    # 2 Create new sets with shorter vectors until done
    # for i in range(80): # To find the shortest vector, unlikely to terminate
    while vectors[0].norm > 72.1: # To find the vector of length 72.08. If it doesn't terminate within 15s, rerun.
        # n-=1
        
        print(f"average norm of vectors: {int(sum([p.norm for p in vectors])/len(vectors))}, shortest = {vectors[0].norm}")
        # print(vectors[0])

        assert (vectors[0].p == np.dot(B, vectors[0].x)).all()

        f = random.choice([find_difference, find_modified_average])
        vectors = augment(vectors, B, n, 0.85, f, timeout=10)
        # vectors = augment(vectors, B, n, 0.8, find_modified_average)
        # vectors = augment(vectors, B, n, 0.8, find_difference)

    return vectors[0]

SV = sieve(B)

print(f"The shortest vector:\n{SV}")

average norm of vectors: 1602, shortest = 546.471408218216
average norm of vectors: 1237, shortest = 415.26617969683014
average norm of vectors: 1072, shortest = 415.26617969683014
average norm of vectors: 980, shortest = 415.26617969683014
average norm of vectors: 900, shortest = 415.26617969683014
average norm of vectors: 831, shortest = 415.26617969683014
average norm of vectors: 783, shortest = 415.26617969683014
average norm of vectors: 716, shortest = 292.95221453336035
average norm of vectors: 671, shortest = 292.95221453336035
average norm of vectors: 639, shortest = 292.95221453336035
average norm of vectors: 590, shortest = 292.95221453336035
average norm of vectors: 544, shortest = 245.2692398161661
average norm of vectors: 513, shortest = 245.2692398161661
average norm of vectors: 472, shortest = 192.4759725264429
average norm of vectors: 443, shortest = 192.4759725264429
average norm of vectors: 408, shortest = 192.4759725264429
average norm of vectors: 373, shortest = 134

In [6]:
write_output(B, SV.p, SV.norm, SV.x, 'SVP output.txt')